Capstone Web Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


all_inactives = []
gamedate = ['202111010PHI']
for date in gamedate:
  url = "https://www.basketball-reference.com/boxscores/"+ date +".html"

  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find the 'scorebox_meta' div
  scorebox_meta = soup.find('div', class_='scorebox_meta')
  # print(scorebox_meta)
  # Extract the text from the last two divs inside 'scorebox_meta' (Game Time and Stadium)
  data = [div.text for div in scorebox_meta.find_all('div')[:2]]
  gametime_stadium = [data]

  # Extract Attendance
  attendance_div = soup.find('strong', string=lambda x: x and 'Attendance:' in x)
  attendance_value = int(attendance_div.next_sibling.strip().replace(",", ""))
  attendance_list = [attendance_value]

  # Extract Records
  scores = soup.find_all('div', class_='score')
  away_record = scores[0].find_next('div').text
  home_record = scores[1].find_next('div').text

  # Extract Inactives
  inactives_strong = soup.find('strong', string=lambda x: x and 'Inactive:' in x)
  inactives_div = inactives_strong.find_parent() if inactives_strong else None

  inactives = {'game_id': date, 'POR': [], 'Opponent': []}
  if inactives_div:
      spans = inactives_div.find_all('span')
      for i, span in enumerate(spans):
          team = span.strong.text if span.strong else 'Unknown'
          players = []
          next_tag = span.find_next_sibling()
          while next_tag and next_tag.name == 'a':
              players.append(next_tag.text)
              next_tag = next_tag.find_next_sibling()
          if team == 'POR':
              inactives['POR'].extend(players)
          else:
              inactives['Opponent'].extend(players)

  all_inactives.append(inactives)

# Convert the collected data to a DataFrame
df_inactives = pd.DataFrame(all_inactives)

# Display the DataFrame
df_inactives


In [ ]:
# import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

gamedates = [
'202110200POR',
'202110230POR',
'202110250LAC',
'202110270POR',
'202110290POR',
'202110310CHO',
'202111010PHI',
'202111030CLE',
'202111050POR',
'202111060POR',
'202111090LAC',
'202111100PHO',
'202111120HOU',
'202111140DEN',
'202111150POR',
'202111170POR',
'202111200POR',
'202111230POR',
'202111240SAC',
'202111260GSW',
'202111290UTA',
'202111300POR',
'202112020POR',
'202112040POR',
'202112060POR',
'202112080GSW',
'202112120POR',
'202112140POR',
'202112150POR',
'202112170CHO',
'202112190MEM',
'202112210NOP',
'202112270POR',
'202112290POR',
'202112310LAL',
'202201030POR',
'202201050POR',
'202201070POR',
'202201090POR',
'202201100POR',
'202201130DEN',
'202201150WAS',
'202201170ORL',
'202201190MIA',
'202201210BOS',
'202201230TOR',
'202201250POR',
'202201260POR',
'202201280HOU',
'202201300CHI',
'202201310OKC',
'202202020LAL',
'202202040POR',
'202202050POR',
'202202080POR',
'202202090POR',
'202202120POR',
'202202140MIL',
'202202160MEM',
'202202240POR',
'202202270POR',
'202203020PHO',
'202203050MIN',
'202203070MIN',
'202203090UTA',
'202203120POR',
'202203140ATL',
'202203160NYK',
'202203180BRK',
'202203200IND',
'202203210DET',
'202203230POR',
'202203250POR',
'202203260POR',
'202203280POR',
'202203300POR',
'202204010SAS',
'202204030SAS',
'202204050OKC',
'202204070NOP',
'202204080DAL',
'202204100POR',

'202210190SAC',
'202210210POR',
'202210230LAL',
'202210240POR',
'202210260POR',
'202210280POR',
'202211020POR',
'202211040PHO',
'202211050PHO',
'202211070MIA',
'202211090CHO',
'202211100NOP',
'202211120DAL',
'202211150POR',
'202211170POR',
'202211190POR',
'202211210MIL',
'202211230CLE',
'202211250NYK',
'202211270BRK',
'202211290POR',
'202211300LAL',
'202212030UTA',
'202212040POR',
'202212080POR',
'202212100POR',
'202212120POR',
'202212140SAS',
'202212160DAL',
'202212170HOU',
'202212190OKC',
'202212210OKC',
'202212230DEN',
'202212260POR',
'202212300GSW',
'202301020POR',
'202301040MIN',
'202301060IND',
'202301080TOR',
'202301100POR',
'202301120POR',
'202301140POR',
'202301150POR',
'202301170DEN',
'202301190POR',
'202301220POR',
'202301230POR',
'202301250POR',
'202301280POR',
'202301300POR',
'202302010MEM',
'202302030WAS',
'202302040CHI',
'202302060POR',
'202302080POR',
'202302100POR',
'202302130POR',
'202302140POR',
'202302230SAC',
'202302260POR',
'202302280GSW',
'202303010POR',
'202303030ATL',
'202303050ORL',
'202303060DET',
'202303080BOS',
'202303100PHI',
'202303120NOP',
'202303140POR',
'202303170POR',
'202303190POR',
'202303220UTA',
'202303240POR',
'202303260POR',
'202303270POR',
'202303290POR',
'202303310POR',
'202304020MIN',
'202304040MEM',
'202304060SAS',
'202304080LAC',
'202304090GSW'

]
all_data = []
all_inactives = []

for date in gamedates:
    url = "https://www.basketball-reference.com/boxscores/" + date + ".html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the 'scorebox_meta' div
    scorebox_meta = soup.find('div', class_='scorebox_meta')
    if not scorebox_meta:
        all_data.append({
          'game_identifier': date,
          'Game Time': 'N/A',
          'Stadium': 'N/A',
          'Attendance': 'N/A',
          'Away Record': 'N/A',
          'Home Record': 'N/A'
        })
        continue

    # Extract the text from the last two divs inside 'scorebox_meta' (game time and stadium)
    game_info = [div.text for div in scorebox_meta.find_all('div')[:2]]

    # Extract Records
    scores = soup.find_all('div', class_='score')
    away_record = scores[0].find_next('div').text
    home_record = scores[1].find_next('div').text

  # Extract Inactives
    inactives_strong = soup.find('strong', string=lambda x: x and 'Inactive:' in x)
    inactives_div = inactives_strong.find_parent() if inactives_strong else None

    inactives = {'game_id': date, 'POR': [], 'Opponent': []}
    if inactives_div:
        spans = inactives_div.find_all('span')
        for i, span in enumerate(spans):
            team = span.strong.text if span.strong else 'Unknown'
            players = []
            next_tag = span.find_next_sibling()
            while next_tag and next_tag.name == 'a':
                players.append(next_tag.text)
                next_tag = next_tag.find_next_sibling()
            if team == 'POR':
                inactives['POR'].extend(players)
            else:
                inactives['Opponent'].extend(players)

    all_inactives.append(inactives)
    # Extract Attendance
    attendance_div = soup.find('strong', string=lambda x: x and 'Attendance:' in x)
    if not attendance_div:
        all_data.append({
            'game_identifier': date,
            'Game Time': game_info[0],
            'Stadium': game_info[1],
            'Attendance': 'Error',
            'Away Record': away_record,
            'Home Record': home_record
        })
        continue
    attendance = int(attendance_div.next_sibling.strip().replace(",", ""))

    # Append data to all_data list
    all_data.append({
        'game_identifier': date,
        'Game Time': game_info[0],
        'Stadium': game_info[1],
        'Attendance': attendance,
        'Away Record': away_record,
        'Home Record': home_record
    })

    time.sleep(4)

# Convert the collected data to a DataFrame
df = pd.DataFrame(all_data)
df2 = pd.DataFrame(all_inactives)

# Display the DataFrame
print(df)
print(df2)


In [ ]:
# df.to_csv('game_data.csv', index=False)
df2.to_csv('inactive_data.csv', index=False)

In [ ]:
pip install basketball-reference-scraper==v1.0.1

In [ ]:
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
s1 = get_team_stats('GSW', 2019, data_format='PER_GAME')
print(s1)

s2 = get_opp_stats('GSW', 2019, data_format='PER_GAME')
print(s2)

s3 = get_injury_report()
print(s3)

# Collecting Additional Data Years

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

gamedates = [
"201710180PHO",
"201710200IND",
"201710210MIL",
"201710240POR",
"201710260POR",
"201710280POR",
"201710300POR",
"201711010UTA",
"201711020POR",
"201711050POR",
"201711070POR",
"201711100POR",
"201711130POR",
"201711150POR",
"201711170SAC",
"201711180POR",
"201711200MEM",
"201711220PHI",
"201711240BKN",
"201711250WAS",
"201711270NYK",
"201711300POR",
"201712020POR",
"201712050POR",
"201712090POR",
"201712110GSW",
"201712130MIA",
"201712150ORL",
"201712160CHO",
"201712180MIN",
"201712200POR",
"201712220POR",
"201712230LAL",
"201712280POR",
"201712300ATL",
"201801010CHI",
"201801020CLE",
"201801050POR",
"201801070POR",
"201801090OKC",
"201801100HOU",
"201801120NOP",
"201801140MIN",
"201801160POR",
"201801180POR",
"201801200POR",
"201801220DEN",
"201801240POR",
"201801260DAL",
"201801300LAC",
"201801310POR",
"201802020TOR",
"201802040BOS",
"201802050DET",
"201802080POR",
"201802090SAC",
"201802110POR",
"201802140POR",
"201802230UTA",
"201802240PHO",
"201802270POR",
"201803010POR",
"201803030POR",
"201803050LAL",
"201803060POR",
"201803090POR",
"201803120POR",
"201803150POR",
"201803170POR",
"201803180LAC",
"201803200POR",
"201803230POR",
"201803250OKC",
"201803270NOP",
"201803280MEM",
"201803300POR",
"201804010POR",
"201804030DAL",
"201804050HOU",
"201804070SAS",
"201804090DEN",
"201804110POR",
"201810180POR",
"201810200POR",
"201810220POR",
"201810250ORL",
"201810270MIA",
"201810290IND",
"201810300HOU",
"201811010POR",
"201811030POR",
"201811040POR",
"201811060POR",
"201811080POR",
"201811110POR",
"201811140LAL",
"201811160MIN",
"201811180WAS",
"201811200NYK",
"201811210MIL",
"201811230GSW",
"201811250POR",
"201811280POR",
"201811300POR",
"201812020SAS",
"201812040DAL",
"201812060POR",
"201812080POR",
"201812110HOU",
"201812120MEM",
"201812140POR",
"201812170LAC",
"201812190POR",
"201812210POR",
"201812230POR",
"201812250UTA",
"201812270GSW",
"201812290POR",
"201812300POR",
"201901010SAC",
"201901040POR",
"201901050POR",
"201901070POR",
"201901090POR",
"201901110POR",
"201901130DEN",
"201901140SAC",
"201901160POR",
"201901180POR",
"201901210UTA",
"201901220OKC",
"201901240PHO",
"201901260POR",
"201901300POR",
"201902050POR",
"201902070POR",
"201902100DAL",
"201902110OKC",
"201902130POR",
"201902210BKN",
"201902230PHI",
"201902250CLE",
"201902270BOS",
"201903010TOR",
"201903030CHO",
"201903050MEM",
"201903070POR",
"201903090POR",
"201903120LAC",
"201903150NOP",
"201903160SAS",
"201903180POR",
"201903200POR",
"201903230POR",
"201903250POR",
"201903270CHI",
"201903290ATL",
"201903300DET",
"201904010MIN",
"201904030POR",
"201904050DEN",
"201904070POR",
"201904090LAL",
"201904100POR",
"201910230POR",
"201910250SAC",
"201910270DAL",
"201910280SAS",
"201910300OKC",
"201911020POR",
"201911040GSW",
"201911070LAC",
"201911080POR",
"201911100POR",
"201911120SAC",
"201911130POR",
"201911160SAS",
"201911180HOU",
"201911190NOP",
"201911210MIL",
"201911230CLE",
"201911250CHI",
"201911270POR",
"201911290POR",
"201912030LAC",
"201912040POR",
"201912060POR",
"201912080POR",
"201912100POR",
"201912120DEN",
"201912160PHO",
"201912180POR",
"201912200POR",
"201912210POR",
"201912230POR",
"201912260UTA",
"201912280POR",
"201912300POR",
"202001010NYK",
"202001030WAS",
"202001050MIA",
"202001070TOR",
"202001090MIN",
"202001110POR",
"202001130POR",
"202001150HOU",
"202001170DAL",
"202001180OKC",
"202001200POR",
"202001230POR",
"202001260POR",
"202001290POR",
"202001310LAL",
"202002010POR",
"202002040DEN",
"202002060POR",
"202002070UTA",
"202002090POR",
"202002110NOP",
"202002120MEM",
"202002210POR",
"202002230POR",
"202002250POR",
"202002270IND",
"202002290ATL",
"202003020ORL",
"202003040POR",
"202003060PHO",
"202003070POR",
"202003100POR"
]
all_data = []
all_inactives = []

for date in gamedates: #parsing through each game_id
    url = "https://www.basketball-reference.com/boxscores/" + date + ".html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the 'scorebox_meta' div
    scorebox_meta = soup.find('div', class_='scorebox_meta')
    if not scorebox_meta: # checking that it collected data
        all_data.append({
          'game_identifier': date,
          'Game Time': 'N/A',
          'Stadium': 'N/A',
          'Attendance': 'N/A',
          'Away Record': 'N/A',
          'Home Record': 'N/A'
        })
        continue

    # Extract the text from the last two divs inside 'scorebox_meta' (game time and stadium)
    game_info = [div.text for div in scorebox_meta.find_all('div')[:2]]

    # Extract Records
    scores = soup.find_all('div', class_='score')
    away_record = scores[0].find_next('div').text
    home_record = scores[1].find_next('div').text

  # Extract Inactives
    inactives_strong = soup.find('strong', string=lambda x: x and 'Inactive:' in x)
    inactives_div = inactives_strong.find_parent() if inactives_strong else None

    inactives = {'game_id': date, 'POR': [], 'Opponent': []}
    if inactives_div:
        spans = inactives_div.find_all('span')
        for i, span in enumerate(spans):
            team = span.strong.text if span.strong else 'Unknown'
            players = []
            next_tag = span.find_next_sibling()
            while next_tag and next_tag.name == 'a':
                players.append(next_tag.text)
                next_tag = next_tag.find_next_sibling()
            if team == 'POR':
                inactives['POR'].extend(players)
            else:
                inactives['Opponent'].extend(players)

    all_inactives.append(inactives)
    # Extract Attendance
    attendance_div = soup.find('strong', string=lambda x: x and 'Attendance:' in x)
    if not attendance_div: # Checking for nul value in attendance
        all_data.append({
            'game_identifier': date,
            'Game Time': game_info[0],
            'Stadium': game_info[1],
            'Attendance': 'Error',
            'Away Record': away_record,
            'Home Record': home_record
        })
        continue
    attendance = int(attendance_div.next_sibling.strip().replace(",", ""))

    # Append data to all_data list
    all_data.append({
        'game_identifier': date,
        'Game Time': game_info[0],
        'Stadium': game_info[1],
        'Attendance': attendance,
        'Away Record': away_record,
        'Home Record': home_record
    })

    time.sleep(4)

# Convert the collected data to a DataFrame
df = pd.DataFrame(all_data)
df2 = pd.DataFrame(all_inactives)

# Display the DataFrame
print(df)
print(df2)


In [ ]:
df.to_csv('game_data2.csv', index=False)
df2.to_csv('inactive_data2.csv', index=False)